# Setup

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# Generate the data


In [2]:
class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))

Generating data...
Total questions: 50000


# Vectorize the data


In [3]:
print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)

Vectorization...


C:\Users\Acer\AppData\Local\Temp/ipykernel_1956/1046822725.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
C:\Users\Acer\AppData\Local\Temp/ipykernel_1956/1046822725.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=n

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


# Build the model


In [4]:
print("Build model...")
num_layers = 1  # Try to add more LSTM layers!

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               72192     
                                                                 
 repeat_vector (RepeatVector  (None, 4, 128)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 4, 128)            131584    
                                                                 
 dense (Dense)               (None, 4, 12)             1548      
                                                                 
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Train the model


In [5]:
epochs = 30
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


Iteration 1
1/1 [==============================] - 1s 712ms/step
Q 580+731 T 1311 ☒ 1270
1/1 [==============================] - 0s 19ms/step
Q 878+657 T 1535 ☒ 1587
1/1 [==============================] - 0s 19ms/step
Q 537+54  T 591  ☒ 557 
1/1 [==============================] - 0s 21ms/step
Q 357+1   T 358  ☒ 547 
1/1 [==============================] - 0s 19ms/step
Q 804+1   T 805  ☒ 158 
1/1 [==============================] - 0s 18ms/step
Q 14+732  T 746  ☒ 597 
1/1 [==============================] - 0s 20ms/step
Q 396+660 T 1056 ☒ 100 
1/1 [==============================] - 0s 18ms/step
Q 3+303   T 306  ☒ 344 
1/1 [==============================] - 0s 23ms/step
Q 25+41   T 66   ☒ 14  
1/1 [==============================] - 0s 18ms/step
Q 69+839  T 908  ☒ 907 

Iteration 2
1/1 [==============================] - 0s 20ms/step
Q 243+955 T 1198 ☒ 1299
1/1 [==============================] - 0s 21ms/step
Q 951+12  T 963  ☒ 969 
1/1 [==============================] - 0s 20ms/step
Q 391+6  

1/1 [==============================] - 0s 26ms/step
Q 56+12   T 68   ☑ 68  
1/1 [==============================] - 0s 23ms/step
Q 918+1   T 919  ☑ 919 
1/1 [==============================] - 0s 29ms/step
Q 16+772  T 788  ☑ 788 
1/1 [==============================] - 0s 28ms/step
Q 681+858 T 1539 ☑ 1539
1/1 [==============================] - 0s 28ms/step
Q 310+6   T 316  ☑ 316 
1/1 [==============================] - 0s 20ms/step
Q 654+46  T 700  ☒ 600 
1/1 [==============================] - 0s 18ms/step
Q 637+620 T 1257 ☑ 1257
1/1 [==============================] - 0s 17ms/step
Q 88+83   T 171  ☑ 171 
1/1 [==============================] - 0s 19ms/step
Q 484+18  T 502  ☑ 502 
1/1 [==============================] - 0s 20ms/step
Q 848+615 T 1463 ☑ 1463

Iteration 11
1/1 [==============================] - 0s 26ms/step
Q 226+780 T 1006 ☑ 1006
1/1 [==============================] - 0s 25ms/step
Q 2+808   T 810  ☑ 810 
1/1 [==============================] - 0s 18ms/step
Q 930+3   T 933  ☑ 933

1/1 [==============================] - 0s 18ms/step
Q 7+15    T 22   ☑ 22  
1/1 [==============================] - 0s 17ms/step
Q 739+617 T 1356 ☑ 1356
1/1 [==============================] - 0s 19ms/step
Q 157+664 T 821  ☑ 821 
1/1 [==============================] - 0s 19ms/step
Q 50+569  T 619  ☑ 619 
1/1 [==============================] - 0s 18ms/step
Q 703+829 T 1532 ☑ 1532
1/1 [==============================] - 0s 22ms/step
Q 69+296  T 365  ☑ 365 
1/1 [==============================] - 0s 17ms/step
Q 379+390 T 769  ☑ 769 
1/1 [==============================] - 0s 18ms/step
Q 441+499 T 940  ☑ 940 
1/1 [==============================] - 0s 21ms/step
Q 3+846   T 849  ☑ 849 
1/1 [==============================] - 0s 20ms/step
Q 455+35  T 490  ☑ 490 

Iteration 20
1/1 [==============================] - 0s 18ms/step
Q 716+953 T 1669 ☑ 1669
1/1 [==============================] - 0s 18ms/step
Q 737+742 T 1479 ☑ 1479
1/1 [==============================] - 0s 18ms/step
Q 820+575 T 1395 ☑ 139

1/1 [==============================] - 0s 19ms/step
Q 51+20   T 71   ☑ 71  
1/1 [==============================] - 0s 18ms/step
Q 758+800 T 1558 ☑ 1558
1/1 [==============================] - 0s 27ms/step
Q 630+729 T 1359 ☒ 1259
1/1 [==============================] - 0s 23ms/step
Q 841+0   T 841  ☑ 841 
1/1 [==============================] - 0s 24ms/step
Q 623+5   T 628  ☑ 628 
1/1 [==============================] - 0s 31ms/step
Q 807+1   T 808  ☑ 808 
1/1 [==============================] - 0s 19ms/step
Q 12+962  T 974  ☑ 974 
1/1 [==============================] - 0s 18ms/step
Q 36+99   T 135  ☑ 135 
1/1 [==============================] - 0s 19ms/step
Q 339+53  T 392  ☑ 392 
1/1 [==============================] - 0s 18ms/step
Q 682+123 T 805  ☑ 805 

Iteration 29
1/1 [==============================] - 0s 21ms/step
Q 150+387 T 537  ☑ 537 
1/1 [==============================] - 0s 20ms/step
Q 26+455  T 481  ☑ 481 
1/1 [==============================] - 0s 19ms/step
Q 182+841 T 1023 ☒ 122